In [1]:
import gensim
import json
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from string import punctuation
import pyLDAvis.gensim 
morph = MorphAnalyzer()

In [2]:
def remove_tags(text):
    return re.sub(r'<[^>]+>', '', text)

# сразу убираем стоп-слова, чтобы неповадно было
stopwds = stopwords.words('russian') + stopwords.words('english')
punctuation = punctuation + '0123456789'
def clean(words):
    clean = [morph.parse(word.strip(punctuation))[0].normal_form for word in words if word not in stopwds \
             and len(word.strip(punctuation)) > 1]
    return clean

## Данные

Возьмем 4 тыс статьи с Хабра. Это мало для хорошей тематической модели, но иначе у нас просто ничего не обучится за семинар.

In [3]:
texts = open('habr_texts.txt').read().splitlines()

В текстах есть тэги. Потрем их. Ещё токенизируем самым простым способом, уберём стоп-слова и нормализуем Pymorphy.

In [4]:
texts = [clean(word_tokenize(text.lower())) for text in texts]

In [73]:
# для нграммов
ph = gensim.models.Phrases(texts, scoring='npmi', threshold=0.4)
p = gensim.models.phrases.Phraser(ph)

In [5]:
len(texts)

4121

### Тематическое моделирование в gensim

Для моделей нужно сделать словарь.

In [6]:
dictionary = gensim.corpora.Dictionary(texts)

In [7]:
dictionary.filter_extremes(no_above=0.3, no_below=10)
dictionary.compactify()

In [8]:
print(dictionary)

Dictionary(15328 unique tokens: ['address', 'api', 'architecture', 'assembly', 'async']...)


Преобразуем наши тексты в мешки слов. 

In [9]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [13]:
# вот это лучшая, вроде как
lsi = gensim.models.LdaModel(corpus, 200, id2word=dictionary, passes=5)

/anaconda3/envs/maga/lib/python3.6/site-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


Посмотрим на топики.

In [84]:
lsi.print_topics()

[(153,
  '0.013*"задача" + 0.008*"пользователь" + 0.008*"проблема" + 0.008*"тестирование" + 0.006*"мы" + 0.006*"тест" + 0.005*"количество" + 0.005*"сервер" + 0.005*"процесс" + 0.004*"запрос"'),
 (53,
  '0.493*"розетка" + 0.021*"нагрузка" + 0.000*"вода" + 0.000*"человек" + 0.000*"research" + 0.000*"смартфон" + 0.000*"поэтому" + 0.000*"компания" + 0.000*"количество" + 0.000*"уровень"'),
 (86,
  '0.093*"gt" + 0.069*"lt" + 0.018*"div" + 0.016*"class" + 0.015*"компонент" + 0.011*"страница" + 0.010*"элемент" + 0.008*"шаблон" + 0.007*"пользователь" + 0.006*"use"'),
 (3,
  '0.047*"игра" + 0.033*"игрок" + 0.029*"человек" + 0.020*"исследование" + 0.017*"ваш" + 0.016*"играть" + 0.015*"понять" + 0.012*"проблема" + 0.011*"вопрос" + 0.009*"тест"'),
 (102,
  '0.114*"датчик" + 0.038*"использование" + 0.037*"решение" + 0.030*"фото" + 0.026*"порог" + 0.020*"оказаться" + 0.019*"алгоритм" + 0.018*"скрипт" + 0.016*"определение" + 0.015*"минус"'),
 (197,
  '0.014*"компания" + 0.009*"человек" + 0.009*"миллио

Можно посмотреть метрики.

In [14]:
lsi.log_perplexity(corpus[:100], total_docs=100)

-20.28027986086004

In [ ]:
# оно почему-то не работает(
coherence_model_lda = gensim.models.CoherenceModel(model=lsi, 
                                                   texts=texts, 
                                                   dictionary=dictinary, coherence='c_v')

In [19]:
topics = []
for topic_id, topic in lda.show_topics(num_topics=10, formatted=False):
    topic = [word for word, _ in topic]
    topics.append(topic)

In [20]:
coherence_model_lda = gensim.models.CoherenceModel(topics=topics, 
                                                   texts=texts, 
                                                   dictionary=dictinary, coherence='c_v')

In [ ]:
coherence_model_lda.get_coherence()

In [11]:
# ещё модель
lsi = gensim.models.LdaModel(corpus, 200, id2word=dictionary, passes=5, eta=0.9)
lsi.print_topics()

[(21,
  '0.000*"книга" + 0.000*"устройство" + 0.000*"сеть" + 0.000*"интернет" + 0.000*"mirai" + 0.000*"сервис" + 0.000*"цензура" + 0.000*"инструкция" + 0.000*"файл" + 0.000*"сообщение"'),
 (140,
  '0.000*"gt" + 0.000*"name" + 0.000*"lt" + 0.000*"user" + 0.000*"conn" + 0.000*"устройство" + 0.000*"язык" + 0.000*"данный" + 0.000*"set" + 0.000*"значение"'),
 (144,
  '0.015*"синтезатор" + 0.011*"инструмент" + 0.008*"устройство" + 0.007*"советский" + 0.007*"фильтр" + 0.005*"производство" + 0.005*"операционный" + 0.005*"усилитель" + 0.004*"создатель" + 0.004*"современный"'),
 (182,
  '0.000*"offer" + 0.000*"webrtc" + 0.000*"боб" + 0.000*"алиса" + 0.000*"кишка" + 0.000*"goes" + 0.000*"answer" + 0.000*"видеоконференция" + 0.000*"договариваться" + 0.000*"распарсить"'),
 (188,
  '0.000*"устройство" + 0.000*"игра" + 0.000*"windows" + 0.000*"ключ" + 0.000*"учётный" + 0.000*"доступ" + 0.000*"клиент" + 0.000*"сеть" + 0.000*"hello" + 0.000*"сервер"'),
 (110,
  '0.000*"язык" + 0.000*"клиент" + 0.000*"э

In [12]:
lsi.log_perplexity(corpus[:100], total_docs=100)

-16.57217908899143

Эта модель похуже, т.к. перплексия больше

А вот три топика для лучшей:

(195,
  '0.029*"ноутбук" + 0.023*"экран" + 0.021*"гб" + 0.019*"видеокарта" + 0.017*"компьютер" + 0.014*"nvidia" + 0.012*"разрешение" + 0.012*"память" + 0.012*"поддержка" + 0.012*"intel"')
   - железо

(60,
  '0.026*"скорость" + 0.015*"расчёт" + 0.013*"параметр" + 0.010*"формула" + 0.009*"слой" + 0.009*"значит" + 0.008*"образ" + 0.008*"вопрос" + 0.007*"сопротивление" + 0.007*"выходной"')
  - машинлёрнинг

(197,
  '0.014*"компания" + 0.009*"человек" + 0.009*"миллион" + 0.008*"китай" + 0.008*"мир" + 0.008*"страна" + 0.007*"сша" + 0.005*"китайский" + 0.005*"рынок" + 0.005*"доллар"')
  - экономика

In [15]:
tfidf = gensim.models.TfidfModel(corpus, id2word=dictionary)
corpus = tfidf[corpus]
lsi = gensim.models.LdaModel(corpus, 200, id2word=dictionary, passes=5)
lsi.print_topics()

/anaconda3/envs/maga/lib/python3.6/site-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


[(68,
  '0.057*"олимпиада" + 0.021*"школьник" + 0.019*"издатель" + 0.014*"ux" + 0.012*"вовлеченность" + 0.011*"доход" + 0.011*"лос-анджелес" + 0.010*"понедельник" + 0.010*"kickstarter" + 0.009*"конкурировать"'),
 (151,
  '0.048*"returns" + 0.041*"invalid" + 0.036*"культовый" + 0.023*"ast" + 0.017*"variable" + 0.012*"дурной" + 0.011*"nmap" + 0.011*"костяк" + 0.010*"redmine" + 0.009*"геймдизайнер"'),
 (181,
  '0.047*"firewall" + 0.011*"демократия" + 0.011*"firmware" + 0.010*"прибегнуть" + 0.010*"винд" + 0.010*"hdmi" + 0.009*"love" + 0.009*"докупать" + 0.009*"virtualbox" + 0.008*"противовес"'),
 (1,
  '0.020*"джобс" + 0.020*"веб-приложение" + 0.016*"hadoop" + 0.012*"доп" + 0.011*"waiting" + 0.008*"потреблять" + 0.008*"упускать" + 0.008*"menu" + 0.007*"safari" + 0.007*"покидать"'),
 (188,
  '0.019*"pm" + 0.007*"threads" + 0.007*"финт" + 0.006*"unified" + 0.005*"translator" + 0.005*"berkeley" + 0.005*"recursive" + 0.001*"thread" + 0.001*"complete" + 0.001*"symbols"'),
 (23,
  '0.037*"pebble

In [16]:
lsi.log_perplexity(corpus[:100], total_docs=100)

-102.44691831317544

Разница:
- уоу, перплексия просто рванула вниз, то есть должно стать лучше
- но топики какие-то вообще непонятные: Джобс и хадуп, демократия и virtualbox, радиоактивный, зомби и видеоматериал?

### Разложение матриц в sklearn

In [17]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd

Sklearn принимает на вход строки, поэтому склеим наши списки.

In [18]:
stexts = [' '.join(text) for text in texts]

Сделаем матрицу слова-документы с помощью TfidfVectorizer

In [30]:
vectorizer = TfidfVectorizer(max_features=20000, min_df=5, max_df=0.7, lowercase=False)
X = vectorizer.fit_transform(stexts)

Разложим её.

In [31]:
model = NMF(n_components=30)

In [32]:
model.fit(X)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
  n_components=30, random_state=None, shuffle=False, solver='cd',
  tol=0.0001, verbose=0)

In [33]:
def get_nmf_topics(model, n_top_words):
    
    #id слов.
    feat_names = vectorizer.get_feature_names()
    
    word_dict = {};
    for i in range(30):
        
        #топ n слов для темы.
        words_ids = model.components_[i].argsort()[:-n_top_words - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic #{:02d}'.format(i+1)] = words; 
    
    return pd.DataFrame(word_dict);

In [34]:
get_nmf_topics(model, 10)

,Topic #01,Topic #02,Topic #03,Topic #04,Topic #05,Topic #06,Topic #07,Topic #08,Topic #09,Topic #10,...,Topic #21,Topic #22,Topic #23,Topic #24,Topic #25,Topic #26,Topic #27,Topic #28,Topic #29,Topic #30
0,проект,gt,игра,дать,сеть,компания,камера,ия,космический,public,...,int,приложение,печать,книга,бот,звук,vr,php,сайт,язык
1,задача,lt,игрок,объект,трафик,год,смартфон,обучение,спутник,string,...,amp,android,принтер,часы,telegram,сигнал,виртуальный,yii,пользователь,программирование
2,команда,div,игровой,функция,оператор,сотрудник,устройство,робот,орбита,void,...,lt,мобильный,станок,порекомендовать,сообщение,наушник,реальность,symfony,реклама,код
3,продукт,class,играть,значение,связь,рынок,гб,нейросеть,ракета,new,...,void,ios,мм,профессиональный,bot,усилитель,oculus,laravel,страница,программист
4,разработка,this,разработчик,элемент,ip,бизнес,процессор,интеллект,марс,return,...,std,google,пластик,работа,телеграм,частота,шлем,gt,товар,лекция
5,система,name,персонаж,запрос,интернет,российский,аккумулятор,искусственный,год,this,...,char,устройство,печатать,каковать,чат,звуковой,технология,версия,клиент,программа
6,разработчик,props,steam,алгоритм,устройство,ит,память,человек,аппарат,var,...,return,app,материал,предпочитать,пользователь,музыка,vive,psr,письмо,java
7,работа,return,год,метод,канал,крупный,цена,машинный,земля,private,...,const,пользователь,производство,каким,api,искажение,unreal,api,скидка,перевод
8,тестирование,text,unity,таблица,dpi,россия,ноутбук,нейронный,луна,class,...,struct,разработчик,деталь,что,мессенджер,акустический,htc,прислать,контент,python
9,клиент,html,геймплей,код,адрес,страна,дисплей,сеть,полёт,null,...,count,play,изделие,работать,канал,устройство,дополнить,форум,рассылка,курс


In [26]:
vectorizer = TfidfVectorizer(max_features=20000, min_df=5, max_df=0.3)
X = vectorizer.fit_transform(stexts)
model = NMF(n_components=30)
model.fit(X)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
  n_components=30, random_state=None, shuffle=False, solver='cd',
  tol=0.0001, verbose=0)

In [27]:
get_nmf_topics(model, 10)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,...,Topic # 21,Topic # 22,Topic # 23,Topic # 24,Topic # 25,Topic # 26,Topic # 27,Topic # 28,Topic # 29,Topic # 30
0,сотрудник,gt,игра,космический,диск,сервер,камера,css,public,объект,...,файл,печать,android,бот,сигнал,язык,запрос,php,товар,vr
1,продукт,lt,игрок,спутник,процессор,центр,устройство,js,void,значение,...,docker,принтер,google,telegram,звук,программирование,null,symfony,скидка,виртуальный
2,бизнес,div,игровой,орбита,память,дата,смартфон,react,int,элемент,...,sudo,станок,ios,сообщение,частота,программист,end,yii,магазин,реальность
3,клиент,class,играть,ракета,intel,инфраструктура,телефон,javascript,string,алгоритм,...,сервер,мм,мобильный,телеграм,наушник,лекция,select,laravel,цена,oculus
4,менеджер,name,персонаж,марс,ssd,услуга,аккумулятор,angular,amp,блок,...,http,пластик,app,чат,усилитель,программа,таблица,gt,рубль,шлем
5,заказчик,html,steam,аппарат,производительность,облачный,датчик,веб,new,метод,...,скрипт,печатать,устройство,bot,искажение,java,go,api,россия,vive
6,деньга,amp,геймплей,земля,накопитель,облако,видео,vue,return,текстура,...,etc,материал,swift,api,звуковой,курс,sql,psr,российский,htc
7,crm,std,unity,луна,ядро,оборудование,xiaomi,html,private,точка,...,nginx,производство,play,мессенджер,музыка,перевод,varchar,прислать,покупка,unreal
8,программист,text,жанр,станция,гб,сервис,экран,браузер,static,изображение,...,amp,деталь,view,канал,устройство,ребёнок,set,форум,рынок,дополнить
9,опыт,type,инди,полёт,тест,клиент,дисплей,es,null,класс,...,usr,изделие,firebase,message,диапазон,python,база,фреймворка,покупатель,дизайн


Темы понятные! Вот код, вот про код/программистов, вот про музыку (25), вот про мобильные приложения (24), вот просто про смартфоны (7), вот про космос (4). Так как нет формальных метрик (мы вроде ничего встроенного не нашли), приходится оценивать на глаз, но мне обе нравятся. 